In [3]:
from waffle_hub.dataset import Dataset
from waffle_hub.hub import Hub
from waffle_utils.file import io

c:\Users\admin\anaconda3\envs\k_public\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# torch avail check
import torch
torch.cuda.is_available()

True

In [4]:
# 학습된 hub를 불러옵니다.
hub = Hub.load(name = "k_project_v2.0.0")

In [23]:
# 이미지에 대한 추론을 실시합니다. 참고로 test 폴더 내에 잡힐 객체가 없어서 임의로 train 데이터 중 객체가 있는 것을 2개 넣어두었습니다.
# 우부툰 및 윈도우에서 정상 동작 확인
hub.inference(source = 'datasets/k_project_raw_dataset/test',confidence_threshold = 0.25, iou_threshold = 0.5)

100%|██████████| 2/2 [00:00<00:00,  3.36it/s]


InferenceResult(predictions=[{'test_1200.png': []}, {'test_7007.png': [{'category_id': 2, 'bbox': [0, 82.34072875976562, 258.0581359863281, 114.78506469726562], 'area': 29621.219834846444, 'iscrowd': 0, 'score': 0.8501087427139282}]}, {'test_8006.png': []}, {'test_8007.png': [{'category_id': 6, 'bbox': [0, 158.2315673828125, 351.9751281738281, 169.5797119140625], 'area': 59687.840836633, 'iscrowd': 0, 'score': 0.7087032794952393}]}, {'test_9200.png': []}, {'test_12000.png': []}, {'test_12001.png': []}, {'test_12009.png': []}], draw_dir=None)

In [5]:
# 실제 답안지와 같은 방식으로 후처리를 진행합니다.
# 성능이 변화되지 않으며 단순히 coco annotation format으로 변환됩니다.
from waffle_utils.file import io

inference_json_path = './hubs/k_project_v2.0.0/inferences/inferences.json'      # 인퍼런스 결과 파일이 입력
answer_json_path = './datasets/k_project_raw_dataset/labels/answer_sample.json'


inference_json = io.load_json(inference_json_path)
answer_json = io.load_json(answer_json_path)


# 키 : 이미지 파일 이름, 값 : 이미지 아이디
images_id_filename_dict = {}
anns_dict = {}
anns_list = []
# 삽입
for answer_id in answer_json["images"] :
    k, v = answer_id['file_name'], answer_id['id']
    images_id_filename_dict[k] = v

anns_count = 0
# inference.json 순회
for infer in inference_json :
    if list(infer.values())[0] == []:
        continue        # 답 없을때 그냥 컨티뉴
    else:
        for anns in list(infer.values())[0]:
            anns_id = int(anns_count)
            image_id = int(images_id_filename_dict[list(infer.keys())[0]])
            category_id = int(anns["category_id"])
            area = float(anns['bbox'][2]*anns['bbox'][3])
            bbox = anns['bbox']
            anns_dict = {
                "id": anns_id,
                "image_id": image_id,
                "category_id": category_id - 1,
                "segmentation": [],
                "area": area,
                "bbox": bbox,
                "iscrowd": 0,
                "attributes": {
                    "occluded": None,
                    "rotation": 0.0
                }
            }
            anns_list.append(anns_dict)
            anns_count += 1
            
            
answer_json['annotations'] = anns_list

#출력이 저장될 부분. 해당 파일을 답지로 제출하였습니다.
result_path = './answer.json'
io.save_json(answer_json,result_path)

In [9]:
# 어노테이션의 values 출력
for info in anns_list:
    print(info)

{'id': 0, 'image_id': 23598, 'category_id': 1, 'segmentation': [], 'area': 29621.219834846444, 'bbox': [0, 82.34072875976562, 258.0581359863281, 114.78506469726562], 'iscrowd': 0, 'attributes': {'occluded': None, 'rotation': 0.0}}
{'id': 1, 'image_id': 27082, 'category_id': 5, 'segmentation': [], 'area': 59687.840836633, 'bbox': [0, 158.2315673828125, 351.9751281738281, 169.5797119140625], 'iscrowd': 0, 'attributes': {'occluded': None, 'rotation': 0.0}}
